In [0]:
# bibliotecas usadas
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 

In [0]:
# para gerar o grafico no browser
%matplotlib inline

In [3]:
# local dos dados e modifica o diretorio
!git clone https://github.com/ucfilho/ANN
%cd ANN
!ls

Cloning into 'ANN'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 72 (delta 5), reused 0 (delta 0), pack-reused 54
Unpacking objects: 100% (72/72), done.
/content/ANN
 ann_aula_ago_14_2018_leitura_git.ipynb
'ANN_aula_Scikit_learn _ago_29_2018.ipynb'
 ANN_Hydrofoil_ago_29_2018.txt.ipynb
 ANN_Hydrofoil_Clean_02_set_12_2018.ipynb
 ANN_Hydrofoil_Neuronicos_02__set_12_2018.ipynb
 ANN_Hydrofoil_Neuronio_set_17_2018.ipynb
 ANN_Hydrofoil_set_05_2018.ipynb
 ANN_Hydrofoil_set_12_2018.ipynb
 Aprendendo_Um_Pouco_mais_sobre_Python_set_05_2018.ipynb
 arvore.jpg
 Aula_003_Python_audio_e_imagem_ago_29_2018.ipynb
 aula_04_rede_neural_set_05_2018.pdf
 BBC_007041008.wav
 Constroi_ANN_set_19_2018.ipynb
 Criando_Gethub_Ver_Youtube.txt
 Differential_Evolution_Python_set_26_2018.ipynb
 FAULT_DIAGNOSIS_maio_25_2017.txt
 Hydrofoil_ago_29_2018.csv
 john_mccarthy.png
 Paper_Carvao_China_abril_12_2016.csv
 Pape

In [0]:
# leitura dos dados
DAT=pd.read_csv("Hydrofoil_ago_29_2018.csv")
#DAT=pd.read_csv("Hydrofoil_ago_29_2018.csv",index_col="emit")

In [0]:
# separa as colunas de interesse
Index=[2,3,4,5,6,7]
Dat_Select=[]
Dat_Select=DAT.iloc[:,Index].copy()

In [0]:
# funcao normatiza dados
def Normatiza(x):
    strings=list(x)
    for i in strings:
        max_x=x[i].max()
        min_x=x[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=(x[i]-a)/b
    return x

In [0]:
# funcao retorna os dados a forma original
def Original(x,x_old):
    strings=list(x)
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=x[i]*b+a
    return x

In [0]:
# separa os dados em normatizados e originais
DAT_OLD=[]
DAT_OLD=Dat_Select.copy()
DAT=Normatiza(Dat_Select)

In [9]:
# separa os dados em treino e teste
train=[]
test=[]
train=DAT.sample(frac=0.8,random_state=None).copy() # poderia ser frac=0.5? sim ou nao?
test=DAT.drop(train.index).copy()
print(train.head())

    Hydrofoil    Froude  Resistence   Sinkage      Trim     Power
42   0.333333  0.728608    0.572917  0.758551  0.371806  0.359479
52   1.000000  0.210026   -0.015625  0.624976  0.511409 -0.404440
16  -1.000000  1.000000    0.993750  0.830109  0.724248  0.993750
25  -0.333333  0.382887    0.281250  0.515567  0.481548 -0.114088
2   -1.000000 -0.896283   -0.980729 -0.898111 -0.462157 -0.999001


In [0]:
# obtem as variaveis independentes (treino e teste)
x_train=train.iloc[:,[0,1]]
x_test=test.iloc[:,[0,1]]
X_OLD=DAT_OLD.iloc[:,[0,1]]

In [0]:
# obtem as variaveis dependentes (treino e teste)
Index=[2,3,4,5]
y_train=train.iloc[:,Index]
y_test=test.iloc[:,Index]
Y_OLD=DAT_OLD.iloc[:,Index]

In [0]:
def rede(i,j):
    clf = MLPRegressor(solver='lbfgs',activation='tanh',alpha=1e-5,
                       hidden_layer_sizes=(i, j),random_state=1)
    return clf

In [14]:
N=10
M=10
test=Original(y_test,Y_OLD)
resist_obs_test=[]
resist_obs_test=test['Resistence'].copy()
Melhor=1e99 #toda vez que fizer o teste e o valor da iteracao der menor que o erro. r^2 nao costuma ser o melhor para calcular o erro. MSE=1/n(yexp-ycal)^2 
ir=0
jr=0
for i in range(1,N): #numero de neuronios da primeira camada oculta
    for j in range(1,M):
        col_names=list(y_train)
        clf=rede(i,j)
        clf.fit(x_train, y_train)
        y_calc_train=[]
        y_calc_test=[]
        y_calc_train=clf.predict(x_train)
        y_calc_test=clf.predict(x_test)
        y_calc_train=pd.DataFrame(y_calc_train)
        y_calc_test=pd.DataFrame(y_calc_test)
        y_calc_train.columns = col_names
        y_calc_test.columns = col_names
        y_calc_train=Original(y_calc_train,Y_OLD)
        y_calc_test=Original(y_calc_test,Y_OLD)
        resist_calc_test=[]
        resist_calc_test=y_calc_test['Resistence'].copy()
        mse=mean_squared_error(resist_obs_test,resist_calc_test)
        R2=r2_score(resist_obs_test,resist_calc_test)
        print("Neuronios=",i,j,"MSE teste=",mse,"R^2 teste=",R2)
        if(mse<Melhor):
            Melhor=mse.copy()
            ix=i
            jx=j
            R2r=R2
print("\n Melhor resposta:")
print("Neuronios=",ix,jx,"MSE teste=",Melhor,"R^2 teste=",R2r)

Neuronios= 1 1 MSE teste= 1760602808.758048 R^2 teste= -2.825330931069622
Neuronios= 1 2 MSE teste= 1760705506.9499085 R^2 teste= -2.825554067468096
Neuronios= 1 3 MSE teste= 1760632344.8619034 R^2 teste= -2.8253951053235196
Neuronios= 1 4 MSE teste= 1760511149.6029608 R^2 teste= -2.8251317796202855
Neuronios= 1 5 MSE teste= 1760766622.4553862 R^2 teste= -2.8256868555292693
Neuronios= 1 6 MSE teste= 1760494455.3536003 R^2 teste= -2.8250955073684563
Neuronios= 1 7 MSE teste= 1760616452.2601235 R^2 teste= -2.8253605748429016
Neuronios= 1 8 MSE teste= 1760655913.7591488 R^2 teste= -2.825446314393047
Neuronios= 1 9 MSE teste= 1760587023.5492084 R^2 teste= -2.8252966339258667
Neuronios= 2 1 MSE teste= 1760685956.0693085 R^2 teste= -2.8255115885012803
Neuronios= 2 2 MSE teste= 1760698013.8698804 R^2 teste= -2.825537786958569
Neuronios= 2 3 MSE teste= 1760644754.7558482 R^2 teste= -2.8254220687879203
Neuronios= 2 4 MSE teste= 1760813120.7632418 R^2 teste= -2.825787884230574
Neuronios= 2 5 MSE